In [45]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [46]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"DXPE"}, 
             {'symbol':"TSM"}, 
             {'symbol':"WLDN"}, 
             {'symbol':"SSRM"}, 
             {'symbol':"LRN", 'basis':137.753}, 
             {'symbol':"UNFI", 'basis':20.752}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':100.482}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':60.671}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':0.010}, 
             {'symbol':"PYPL", 'basis':68.738}, 
             {'symbol':"ALL", 'basis':194.927}, 
             {'symbol':"LC", 'basis':8.752}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':0.010}, 
             {'symbol':"CCL", 'basis':8.852}, 
             {'symbol':"AGX", 'basis':0.010}, 
             {'symbol':"POWL", 'basis':164.655}, 
             {'symbol':"PPC", 'basis':45.089}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE", 'basis':115.21}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':73.160}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA", 'basis':97.398}, 
             {'symbol':"TWLO", 'basis':87.178}, 
             {'symbol':"PEP"}, 
             {'symbol':"APP", 'basis':221.323}, 
             {'symbol':"TMUS", 'basis':225.751}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':40.612}, 
             {'symbol':"CAAP", 'basis':21.570}
             ]

# Brokerage
brokerage_funds = [{'symbol':"ALAB"}, 
                   {'symbol':"MU"}, 
                   {'symbol':"STX"}, 
                   {'symbol':"AMZN"}, 
                   {'symbol':"WLDN"}, 
                   {'symbol':"SSRM"}, 
                   {'symbol':'BCS', 'basis':18.569}, 
                   {'symbol':'PUK'}, # Hong Kong - Financials
                   {'symbol':'FINV'}, # China - Financials
                   {'symbol':'PSIX'}, # U.S. - Industrials
                   {'symbol':'NGD'}, # Canada - Materials (Gold)
                   {'symbol':'GFI'}, # South Africa - Materials (Gold)
                   {'symbol':"CRDO", 'basis':0.010}, 
                   {'symbol':"COMM"}, 
                   {'symbol':'BKTI', 'basis':40.200}, 
                   {'symbol':"UNFI", 'basis':22.716}, 
                   {'symbol':"MFC"}, 
                   {'symbol':"EZPW", 'basis':17.715}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"WFC", 'basis':66.893}, 
                   {'symbol':"ITRN"}, 
                   {'symbol':"PYPL", 'basis':68.030}, 
                   {'symbol':"LC"}, 
                   {'symbol':"CLS", 'basis':70.729}, 
                   {'symbol':"CCL", 'basis':0.010}, 
                   {'symbol':"PPC", 'basis':44.580}, 
                   {'symbol':"SYF"}, 
                   {'symbol':"ATGE", 'basis':0.010}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"BLBD"}, 
                   {'symbol':"GM"}, 
                   {'symbol':"OKTA", 'basis':99.535}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   #{'symbol':"GRBK"}, 
                   #{'symbol':"CAAP"},
                   {'symbol':"FBTC", 'basis':65.965, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':58.908},
                   {'symbol':"QUBT"},
                   {'symbol':"RGTI"}
                   ]

In [47]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [48]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=.99, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [49]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
6,BCS,18.569,19.01,17.08,45.16,17.51,9.32,0.652,0.0227,Sell
12,CRDO,0.010,98.45,68.77,6.19,73.99,348.93,24.860,0.0000,Sell
14,BKTI,40.200,41.40,45.03,29.12,49.58,15.56,4.456,0.0000,Hold
15,UNFI,22.716,27.37,24.02,24.02,25.56,inf,0.872,0.0000,Sell
17,EZPW,17.715,14.23,14.34,23.88,14.91,11.95,1.019,0.0000,Hold
18,ARQT,15.996,15.48,13.50,13.50,14.41,inf,12.890,0.0000,Sell
19,WFC,66.893,79.71,73.65,99.10,76.01,14.37,1.544,0.0200,Sell
21,PYPL,68.030,73.86,68.77,56.70,71.34,16.40,3.548,0.0000,Sell
23,CLS,70.729,162.89,114.17,41.17,121.13,45.37,11.830,0.0000,Sell
24,CCL,0.010,29.45,23.10,22.20,24.10,15.20,3.972,0.0000,Sell


In [50]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,ALAB,0.0,97.95,83.13,17.94,89.47,70.18,14.560,0.0000,Sell
1,MU,0.0,113.26,101.53,89.53,106.00,20.98,2.616,0.0037,Sell
2,STX,0.0,146.72,113.65,18.55,118.28,21.17,101.310,0.0200,Sell
3,AMZN,0.0,223.88,200.25,77.57,206.96,36.35,7.860,0.0000,Sell
4,WLDN,0.0,79.67,58.34,31.98,61.42,47.14,4.516,0.0000,Sell
5,SSRM,0.0,12.12,11.50,14.48,12.14,30.27,0.794,0.0000,Hold
7,PUK,0.0,24.70,23.05,27.95,23.58,14.74,1.817,0.0185,Sell
8,FINV,0.0,10.13,8.78,19.77,9.27,7.23,1.245,0.0292,Sell
9,PSIX,0.0,84.03,49.38,22.20,55.03,23.80,20.640,0.0000,Sell
10,NGD,0.0,4.34,4.17,2.78,4.40,24.61,3.393,0.0000,Hold


In [51]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [52]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
4,LRN,137.753,132.75,135.95,83.66,149.19,20.83,4.145,0.0000,Hold
5,UNFI,20.752,27.37,23.05,23.05,25.56,inf,0.872,0.0000,Sell
7,EAT,100.482,165.13,148.03,37.91,162.70,22.93,28.380,0.0000,Sell
8,EZPW,15.460,14.23,13.76,23.88,14.91,11.95,1.019,0.0000,Hold
9,ARQT,15.932,15.48,12.96,12.96,14.41,inf,12.890,0.0000,Sell
10,WFC,60.671,79.71,70.67,99.10,76.01,14.37,1.544,0.0200,Sell
11,ITRN,35.710,38.90,34.68,30.16,37.66,13.91,4.102,0.0513,Sell
12,CRDO,0.010,98.45,65.99,6.19,73.99,348.93,24.860,0.0000,Sell
13,PYPL,68.738,73.86,65.99,56.70,71.34,16.40,3.548,0.0000,Sell
14,ALL,194.927,193.02,186.58,193.43,200.71,13.18,2.591,0.0199,Hold


In [53]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,DXPE,0.0,100.41,80.87,72.41,89.22,20.26,3.255,0.0000,Sell
1,TSM,0.0,245.60,184.33,96.61,198.55,27.95,7.930,0.0145,Sell
2,WLDN,0.0,79.67,55.98,31.98,61.42,47.14,4.516,0.0000,Sell
3,SSRM,0.0,12.12,11.04,14.48,12.14,30.27,0.794,0.0000,Hold
6,MFC,0.0,30.91,29.37,36.24,31.49,15.78,1.581,0.0402,Hold
16,QTWO,0.0,91.44,82.14,82.14,89.15,inf,10.000,0.0000,Sell
22,SYF,0.0,70.19,55.40,102.37,59.82,9.49,1.699,0.0180,Sell
24,BRK-B,0.0,473.40,472.20,627.51,504.62,12.56,1.554,0.0000,Hold
25,SFM,0.0,169.61,152.96,45.80,167.49,37.59,12.510,0.0000,Sell
27,BLBD,0.0,42.86,37.21,25.83,40.48,13.30,7.100,0.0000,Sell


In [54]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'}, # Mexico - Communications - mobile network
              {'symbol':'ERIC'}, # Sweden - Communications
              {'symbol':'RDWR'}, # Sweden - Communications
              {'symbol':'PGY'}, # Sweden - Communications
              {'symbol':'STX'} # Ireland - Technology
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)
internationals = build_analysis_table(watch_list, api_key, margin_of_safety=0.95, vwap_days=126)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

internationals_df = pd.DataFrame(internationals, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [55]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.58,24.15,28.32,24.68,10.08,2.3580,0.0535,Hold
1,KARO,0,50.42,48.20,20.47,51.02,30.16,6.9000,0.0221,Hold
2,QFIN,0,42.00,41.31,73.18,42.94,6.36,1.7760,0.0300,Hold
3,TSM,0,245.60,194.03,96.61,198.55,27.95,7.9300,0.0145,Sell
4,CCU,0,12.04,13.69,16.37,14.05,12.78,1.4520,0.0353,Hold
5,ABEV,0,2.38,2.41,2.45,2.47,15.12,2.1490,0.0634,Hold
6,TIMB,0,18.56,18.02,197.86,18.41,14.72,0.0205,0.0768,Sell
7,ASR,0,314.82,312.41,284.36,320.37,12.97,3.2410,0.0771,Hold
8,RYAAY,0,58.04,53.35,42.53,54.42,17.23,3.7070,0.0162,Sell
9,AMX,0,17.36,16.97,10.51,17.33,35.73,2.6200,0.0305,Sell


In [56]:
internationals_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,23.58,21.95,28.32,23.66,10.08,2.3580,0.0535,Hold
1,KARO,0,50.42,43.97,20.47,48.95,30.16,6.9000,0.0221,Sell
2,QFIN,0,42.00,38.85,73.18,43.02,6.36,1.7760,0.0300,Hold
3,TSM,0,245.60,175.51,96.61,191.58,27.95,7.9300,0.0145,Sell
4,CCU,0,12.04,13.07,16.37,14.17,12.78,1.4520,0.0353,Hold
5,ABEV,0,2.38,2.11,2.45,2.28,15.12,2.1490,0.0634,Sell
6,TIMB,0,18.56,14.97,197.86,16.12,14.72,0.0205,0.0768,Sell
7,ASR,0,314.82,281.22,284.36,304.82,12.97,3.2410,0.0771,Sell
8,RYAAY,0,58.04,46.38,42.53,50.05,17.23,3.7070,0.0162,Sell
9,AMX,0,17.36,14.91,10.51,16.09,35.73,2.6200,0.0305,Sell


In [57]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, margin_of_safety=0.95, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [58]:
mag7_test_table

[['MSFT', 0, 511.7, 427.54, 140.28, 457.09, 39.07, 11.68, 0.0067, 'Sell'],
 ['AAPL', 0, 210.02, 191.52, 31.46, 206.0, 32.68, 46.76, 0.0051, 'Sell'],
 ['NVDA', 0, 173.0, 125.62, 19.76, 136.0, 55.1, 47.72, 0.0003, 'Sell'],
 ['AMZN', 0, 223.88, 192.16, 77.57, 206.96, 36.35, 7.86, 0, 'Sell'],
 ['GOOG', 0, 184.7, 158.88, 94.31, 172.27, 20.49, 6.42, 0.0047, 'Sell'],
 ['GOOGL', 0, 183.58, 157.37, 94.09, 170.76, 20.4, 6.4, 0.0048, 'Sell'],
 ['META', 0, 701.41, 599.59, 250.35, 646.67, 27.47, 9.8, 0.0028, 'Sell'],
 ['BRK-B', 0, 473.4, 472.2, 627.51, 504.62, 12.56, 1.554, 0, 'Hold'],
 ['TSLA', 0, 319.41, 286.91, 37.21, 317.79, 184.87, 13.67, 0, 'Sell']]

In [59]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,511.70,427.54,140.28,457.09,39.07,11.680,0.0067,Sell
1,AAPL,0,210.02,191.52,31.46,206.00,32.68,46.760,0.0051,Sell
2,NVDA,0,173.00,125.62,19.76,136.00,55.10,47.720,0.0003,Sell
3,AMZN,0,223.88,192.16,77.57,206.96,36.35,7.860,0.0000,Sell
4,GOOG,0,184.70,158.88,94.31,172.27,20.49,6.420,0.0047,Sell
5,GOOGL,0,183.58,157.37,94.09,170.76,20.40,6.400,0.0048,Sell
6,META,0,701.41,599.59,250.35,646.67,27.47,9.800,0.0028,Sell
7,BRK-B,0,473.40,472.20,627.51,504.62,12.56,1.554,0.0000,Hold
8,TSLA,0,319.41,286.91,37.21,317.79,184.87,13.670,0.0000,Sell


In [60]:
# Machine Learning

In [61]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [62]:
# Feature selection and normalization

In [63]:
# Build model

In [64]:
# Train model

In [65]:
# Evaluate model

In [66]:
# Refine model through hyperparameter tuning